In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('data.csv')
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
data.shape

(8523, 12)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
Item_Identifier              8523 non-null object
Item_Weight                  7060 non-null float64
Item_Fat_Content             8523 non-null object
Item_Visibility              8523 non-null float64
Item_Type                    8523 non-null object
Item_MRP                     8523 non-null float64
Outlet_Identifier            8523 non-null object
Outlet_Establishment_Year    8523 non-null int64
Outlet_Size                  6113 non-null object
Outlet_Location_Type         8523 non-null object
Outlet_Type                  8523 non-null object
Item_Outlet_Sales            8523 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


### Пропущены значения в весе товара и размере магазина. Заполним их.

In [5]:
(data.Item_Weight == 0).sum() 
#в весе нет нулей, только пропущенные

0

In [6]:
data.groupby('Item_Identifier')['Item_Weight'].mean().isna().sum() 
#есть вес всех товаров, кроме 4

4

In [7]:
data.groupby('Item_Type')['Item_Weight'].mean().isna().sum() 
#можно использовать средний вес товаров в категории для товаров без веса

0

In [8]:
#заполним пропуски веса средним весом товара по ID. 
#Если его нет, то средним весом товара в категории.
for r in range(0, data.shape[0]):
    if data.Item_Weight.isna()[r]:
        if data[data.Item_Identifier == data.Item_Identifier[r]].Item_Weight.mean() >0 :
            data.Item_Weight[r] = data[data.Item_Identifier == data.Item_Identifier[r]].Item_Weight.mean()
        else:
            data.Item_Weight[r] = data[data.Item_Type == data.Item_Type[r]].Item_Weight.mean() 

C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
data.fillna("NA").groupby(['Outlet_Identifier','Outlet_Size','Outlet_Type']).sum()
#не известны размеры 3-х магазинов

,,,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Outlet_Identifier,Outlet_Size,Outlet_Type,,,,,
OUT010,NA,Grocery Store,7166.800000,56.308832,78131.5646,1108890,1.883402e+05
OUT013,High,Supermarket Type1,12121.730000,55.879859,131809.0156,1851884,2.142664e+06
OUT017,NA,Supermarket Type1,11877.495000,56.834646,129103.9564,1858482,2.167465e+06
OUT018,Medium,Supermarket Type2,11946.465000,56.621454,131477.7724,1864352,1.851823e+06
OUT019,Small,Grocery Store,6868.005676,57.257041,73807.5824,1048080,1.796941e+05
OUT027,Medium,Supermarket Type3,12094.283399,54.804764,130714.6746,1855975,3.453926e+06
OUT035,Small,Supermarket Type1,11931.295000,56.974873,133103.9070,1863720,2.268123e+06
OUT045,NA,Supermarket Type1,11751.840000,56.180780,130942.7782,1859858,2.036725e+06
OUT046,Small,Supermarket Type1,11966.125000,56.231875,132113.3698,1857210,2.118395e+06


In [10]:
data.fillna("NA").groupby(['Outlet_Identifier','Outlet_Size','Outlet_Type']).sum().groupby('Outlet_Size').mean()
#гипотеза о том, что по продажам магазина можно сделать вывод о его размере не подтвердилась. Продажи Medium выше, чем в High

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Outlet_Size,,,,,
High,12121.730000,55.879859,131809.015600,1.851884e+06,2.142664e+06
Medium,12017.991133,55.991791,130889.768933,1.859799e+06,2.496573e+06
NA,10265.378333,56.441419,112726.099733,1.609077e+06,1.464177e+06
Small,10255.141892,56.821263,113008.286400,1.589670e+06,1.522071e+06


In [11]:
# посмотрим, как кол-во sku соотноситься с размером магазина
data.fillna("NA").groupby(['Outlet_Identifier','Outlet_Size','Outlet_Type','Item_Identifier']).sum().groupby(['Outlet_Identifier','Outlet_Size','Outlet_Type']).count()

,,,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Outlet_Identifier,Outlet_Size,Outlet_Type,,,,,
OUT010,NA,Grocery Store,555,555,555,555,555
OUT013,High,Supermarket Type1,932,932,932,932,932
OUT017,NA,Supermarket Type1,926,926,926,926,926
OUT018,Medium,Supermarket Type2,928,928,928,928,928
OUT019,Small,Grocery Store,528,528,528,528,528
OUT027,Medium,Supermarket Type3,935,935,935,935,935
OUT035,Small,Supermarket Type1,930,930,930,930,930
OUT045,NA,Supermarket Type1,929,929,929,929,929
OUT046,Small,Supermarket Type1,930,930,930,930,930


OUT010 однозначно Small, OUT017 и OUT045 присвоим Medium

In [12]:
for r in range(0, data.shape[0]):
    if data.Outlet_Identifier[r] == 'OUT010':
        data.Outlet_Size[r] = 'Small'
    elif data.Outlet_Identifier[r] == 'OUT017':
        data.Outlet_Size[r] = 'Medium'
    elif data.Outlet_Identifier[r] == 'OUT045':
        data.Outlet_Size[r] = 'Medium'
        

C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
Item_Identifier              8523 non-null object
Item_Weight                  8523 non-null float64
Item_Fat_Content             8523 non-null object
Item_Visibility              8523 non-null float64
Item_Type                    8523 non-null object
Item_MRP                     8523 non-null float64
Outlet_Identifier            8523 non-null object
Outlet_Establishment_Year    8523 non-null int64
Outlet_Size                  8523 non-null object
Outlet_Location_Type         8523 non-null object
Outlet_Type                  8523 non-null object
Item_Outlet_Sales            8523 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [14]:
data.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

Пустых и нулевых значение больше нет.

### Обрабатываем категориальные признаки .

In [15]:
data.Item_Identifier.value_counts()

FDW13    10
FDG33    10
FDW49     9
FDT07     9
NCQ06     9
         ..
FDC23     1
FDE52     1
FDO33     1
FDK57     1
FDQ60     1
Name: Item_Identifier, Length: 1559, dtype: int64

In [16]:
#1559 вариантов id товара слишком много для категориального признака - удалим его

In [17]:
data.drop(columns = ['Item_Identifier'], axis=1, inplace=True)
data.shape

(8523, 11)

In [18]:
data_dummies = pd.get_dummies(data)
data_dummies.shape

(8523, 46)

In [19]:
data_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 46 columns):
Item_Weight                        8523 non-null float64
Item_Visibility                    8523 non-null float64
Item_MRP                           8523 non-null float64
Outlet_Establishment_Year          8523 non-null int64
Item_Outlet_Sales                  8523 non-null float64
Item_Fat_Content_LF                8523 non-null uint8
Item_Fat_Content_Low Fat           8523 non-null uint8
Item_Fat_Content_Regular           8523 non-null uint8
Item_Fat_Content_low fat           8523 non-null uint8
Item_Fat_Content_reg               8523 non-null uint8
Item_Type_Baking Goods             8523 non-null uint8
Item_Type_Breads                   8523 non-null uint8
Item_Type_Breakfast                8523 non-null uint8
Item_Type_Canned                   8523 non-null uint8
Item_Type_Dairy                    8523 non-null uint8
Item_Type_Frozen Foods             8523 non-null uint8
Item_

### Смотрим корреляцию признаков с продажами

In [20]:
corr = data_dummies.corr()
corr['Item_Outlet_Sales'].sort_values(0)[:-1]

Outlet_Type_Grocery Store         -0.411727
Outlet_Identifier_OUT010          -0.284883
Outlet_Identifier_OUT019          -0.277250
Outlet_Size_Small                 -0.240778
Item_Visibility                   -0.128625
Outlet_Location_Type_Tier 1       -0.111287
Outlet_Establishment_Year         -0.049135
Item_Type_Baking Goods            -0.038381
Outlet_Type_Supermarket Type2     -0.038059
Outlet_Identifier_OUT018          -0.038059
Item_Type_Health and Hygiene      -0.025587
Item_Type_Soft Drinks             -0.024040
Item_Type_Others                  -0.021267
Item_Fat_Content_reg              -0.015148
Item_Fat_Content_LF               -0.012389
Item_Fat_Content_Low Fat          -0.011993
Item_Type_Frozen Foods            -0.009482
Item_Fat_Content_low fat          -0.006326
Item_Type_Breakfast               -0.004656
Item_Type_Hard Drinks             -0.003956
Item_Type_Meat                    -0.002995
Outlet_Identifier_OUT045           0.002274
Item_Type_Breads                

1) Продажи отрицательно коррелируют с Outlet_Type_Grocery Store -> продажи меньше в Grocery Store

2) Продажи положительно коррелируют с Item_MRP -> чем выше цена, тем больше продажи в деньгах

3) Продажи положительно коррелируют с Outlet_Type_Supermarket Type3 -> продажи в большом формате магазина выше

4) С категорий товара корреляция очень слабая

### Создаем отложенную выборку

In [21]:
from sklearn.model_selection import train_test_split
X = data_dummies.drop(columns = ['Item_Outlet_Sales'], axis=1)
y = data_dummies.Item_Outlet_Sales

In [22]:
X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.2)

### Номализация

In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_hold_scaled = sc.transform(X_hold)

C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


### Метрикой возьмем среднеквадратичное отклонение прогноза продаж от факта

In [24]:
from sklearn.metrics import mean_squared_error as mse

# Линейная модель - Стохастический градиентый спуск

In [25]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV

In [26]:
#Подбираем параметры модели

params = {'alpha':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 0.7, 1], 
         'penalty':['l1', 'l2'],
         'fit_intercept' : [True, False]
         }

sgd_grid = GridSearchCV(SGDRegressor(),param_grid=params, scoring='neg_mean_squared_error', verbose=1,n_jobs=-1,cv=3)
sgd_grid.fit(X_train_scaled,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    2.7s finished
C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 0.7, 1], 'penalty': ['l1', 'l2'], 'fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [27]:
#лучшая модель
sgd_grid.best_params_

{'alpha': 0.2, 'fit_intercept': True, 'penalty': 'l1'}

In [28]:
#ошибка лучшей модели
-sgd_grid.best_score_

1304536.077143067

# Не линейная модель - Decision Tree Regressor

In [29]:
from sklearn.tree import DecisionTreeRegressor

In [30]:
#Подбираем параметры модели
dt_params = {
    'criterion' : ['mse', 'mae'],
    'max_depth' : [2, 3, 4, 5, 7, 9, None],
    'min_samples_leaf' : [1, 2, 4, 8, 16]
}

dt_grid = GridSearchCV(DecisionTreeRegressor(),param_grid=dt_params, scoring='neg_mean_squared_error', verbose=1,n_jobs=-1,cv=3)
dt_grid.fit(X_train_scaled,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 70 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   26.1s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'criterion': ['mse', 'mae'], 'max_depth': [2, 3, 4, 5, 7, 9, None], 'min_samples_leaf': [1, 2, 4, 8, 16]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [31]:
#лучшая модель
dt_grid.best_params_

{'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 16}

In [32]:
#ошибка лучшей модели
-dt_grid.best_score_

1199893.5182325852

# Не линейная модель - Random Forest

In [33]:
from sklearn.ensemble import RandomForestRegressor

In [34]:
#Подбираем параметры модели
rf_params = {
    'n_estimators' : [5, 10, 20, 50],
    'max_depth' : [2, 3, 4, 5, 7, 9, None],
    'min_samples_leaf' : [1, 2, 4, 8, 16]
}

rf_grid = GridSearchCV(RandomForestRegressor(),param_grid=rf_params, scoring='neg_mean_squared_error', verbose=1,n_jobs=-1,cv=3)
rf_grid.fit(X_train_scaled,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 140 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 420 out of 420 | elapsed:   14.7s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [5, 10, 20, 50], 'max_depth': [2, 3, 4, 5, 7, 9, None], 'min_samples_leaf': [1, 2, 4, 8, 16]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [35]:
#лучшая модель
rf_grid.best_params_

{'max_depth': 5, 'min_samples_leaf': 8, 'n_estimators': 50}

In [36]:
#ошибка лучшей модели
-rf_grid.best_score_

1191912.9345025553

# Стекинг нескольких моделей: Стохастического Градиентного спуска и Random forest

In [37]:
#Берем лучшие модели
sgd = SGDRegressor(alpha=0.5, fit_intercept=True, penalty='l1')
rf = RandomForestRegressor(max_depth=5, min_samples_leaf=8, n_estimators=10)

In [59]:
def get_meta_features(clf, X_train_, y_train_, X_test_, y_test_, stack_cv):
    meta_train = np.zeros_like(y_train_, dtype=float)
    meta_test = np.zeros_like(y_test_, dtype=float)
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train_, y_train_)):
        
        clf.fit(X_train_.iloc[train_ind], y_train_.iloc[train_ind])
        meta_train[test_ind] = clf.predict(X_train_.iloc[test_ind])[0]
        
        meta_test += clf.predict(X_test_)[0]
    
    return meta_train , meta_test / stack_cv.n_splits

In [60]:
from sklearn.model_selection import KFold

stack_cv = KFold(n_splits=10, shuffle=True, random_state=1)

meta_train = []
meta_test = []
col_names = []

print('SGD features...')
meta_tr, meta_te = get_meta_features(sgd, pd.DataFrame(X_train_scaled), pd.DataFrame(y_train), pd.DataFrame(X_hold_scaled), pd.DataFrame(y_hold), stack_cv)

meta_train.append(meta_tr.reshape(meta_tr.shape[0],))
meta_test.append(meta_te.reshape(meta_te.shape[0],))
col_names.append('sgd_pred')

print('RF features...')
meta_tr, meta_te = get_meta_features(rf, pd.DataFrame(X_train_scaled), pd.DataFrame(y_train), pd.DataFrame(X_hold_scaled), pd.DataFrame(y_hold), stack_cv)


meta_train.append(meta_tr.reshape(meta_tr.shape[0],))
meta_test.append(meta_te.reshape(meta_te.shape[0],))
col_names.append('rf_pred')

SGD features...


C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sidorov\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and 

RF features...


C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
C:\Users\sidorov\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
C:\Users\sidorov\Anaconda3\lib\site-packages\ipy

In [61]:
X_meta_train = pd.DataFrame(np.stack(meta_train, axis=1), columns=col_names)
X_meta_test = pd.DataFrame(np.stack(meta_test, axis=1), columns=col_names)

In [41]:
#pd.concat([X_meta_train.reset_index(), y_train.reset_index()], axis=1, ignore_index=True)

In [62]:
from sklearn.linear_model import LinearRegression

In [63]:
clf_lr_meta = LinearRegression()
clf_lr_meta.fit(X_meta_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [65]:
#прогноз стекинга и ошибка на отложенной выборке

y_pred_meta_test = clf_lr_meta.predict(X_meta_test)

np.sqrt(mse(y_hold, y_pred_meta_test))

1671.8840164936591

In [45]:
#коэффициенты ткенига моделей
clf_lr_meta.coef_

array([-0.08620243,  0.07466826])

# Ошибка моделей на отложеной выборке

In [66]:
estimators = {
    'SGD Regression' : sgd_grid,
    'Decision Tree' : dt_grid,
    'Random Forest' : rf_grid,
}

In [67]:
for k in estimators:
    v = estimators[k]
    print(k, 'CV MSE:', np.round(np.sqrt(-v.best_score_)), 'Validation MSE:',  np.round(np.sqrt(mse(y_hold, v.best_estimator_.predict(X_hold_scaled)))))          

SGD Regression CV MSE: 1142.0 Validation MSE: 1115.0
Decision Tree CV MSE: 1095.0 Validation MSE: 1056.0
Random Forest CV MSE: 1092.0 Validation MSE: 1049.0


# Определим наиболее влияющие признаки

значимость фич возьмем из RandomForest

In [76]:
fi = rf_grid.best_estimator_.feature_importances_

In [79]:
fi_df = pd.DataFrame(list(zip(X_train.columns,fi)), columns=['Feature', 'Feature Importances'])

In [82]:
fi_df.sort_values('Feature Importances', ascending=False).head()

,Feature,Feature Importances
2,Item_MRP,0.550143
41,Outlet_Type_Grocery Store,0.317226
44,Outlet_Type_Supermarket Type3,0.059079
30,Outlet_Identifier_OUT027,0.054305
3,Outlet_Establishment_Year,0.013445


### Самые значимые признаки:
цена - Item_MRP

признак продуктовый магазин - Outlet_Type_Grocery Store

признак типа магазин Supermarket Type3 - Outlet_Type_Supermarket Type3

In [83]:
#сохраним обраборанный DataFrame
data_dummies.to_csv('final_data.csv')